In [10]:
import requests
from bs4 import BeautifulSoup as bs
import json
import random
import os.path

insta_url = 'https://www.instagram.com'
inta_username = input('enter username of instagram : ')

response = requests.get(f"{insta_url}/{inta_username}/")

if response.ok:
    html = response.text
    bs_html = bs(html, features="lxml")
    bs_html = bs_html.text
    index = bs_html.find('profile_pic_url_hd')+21
    remaining_text = bs_html[index:]
    remaining_text_index = remaining_text.find('requested_by_viewer')-3
    string_url = remaining_text[:remaining_text_index].replace("\\u0026", "&")

    print(string_url, "\n \n downloading..........")


while True:
    filename = 'pic'+str(random.randint(1, 100000))+'.jpg'
    file_exists = os.path.isfile(filename)

    if not file_exists:
        with open(filename, 'wb+') as handle:
            response = requests.get(string_url, stream=True)
            if not response.ok:
                print(response)
            for block in response.iter_content(1024):
                if not block:
                    break
                handle.write(block)
    else:
        continue
    break
print("\n                downloading completed ..............")

enter username of instagram : rayanna_na_abhimani_official


InvalidSchema: No connection adapters were found for 'ps://instagram.fixe1-2.fna.fbcdn.net/v/t51.2885-19/s320x320/145898813_418707929209245_4130442731032398024_n.jpg?_nc_ht=instagram.fixe1-2.fna.fbcdn.net&_nc_ohc=bU5wEGSaPpAAX8XVf8q&tp=1&oh=cc64185788f861f445dd692b77aca5dd&oe=604FE64F'

In [12]:
# importing libraries 
from bs4 import BeautifulSoup 
import requests 
  
# instagram URL 
URL = "https://www.instagram.com/{}/"
  
# parse function 
def parse_data(s): 
      
    # creating a dictionary 
    data = {} 
      
    # splittting the content  
    # then taking the first part 
    s = s.split("-")[0] 
      
    # again splitting the content  
    s = s.split(" ") 
      
    # assigning the values 
    data['Followers'] = s[0] 
    data['Following'] = s[2] 
    data['Posts'] = s[4] 
      
    # returning the dictionary 
    return data 
  
# scrape function 
def scrape_data(username): 
      
    # getting the request from url 
    r = requests.get(URL.format(username)) 
      
    # converting the text 
    s = BeautifulSoup(r.text, "html.parser") 
      
    # finding meta info 
    meta = s.find("meta", property ="og:description") 
      
    # calling parse method 
    return parse_data(meta.attrs['content']) 
  
# main function 
if __name__=="__main__": 
      
    # user name 
    username = "rayanna_na_abhimani_official"
      
    # calling scrape function 
    data = scrape_data(username) 
      
    # printing the info 
    print(data) 

AttributeError: 'NoneType' object has no attribute 'attrs'

In [14]:
import argparse
import os
import requests
import time
import urllib.request
from colorama import init, deinit
from termcolor import cprint

def print_in_color(text, color):
    """ Prints `text` in passed `color` """
    cprint(text, color)

def image_downloader(edge, images_path):
    """ Downloads images """
    display_url = edge['node']['display_url']
    file_name = edge['node']['taken_at_timestamp']
    download_path = f"{images_path}\\{file_name}.jpg"
    if not os.path.exists(download_path):
        print_in_color(f"Downloading {str(file_name)}.jpg...........", "yellow")
        urllib.request.urlretrieve(display_url, download_path)
        print_in_color(f"{file_name}.jpg downloaded.\n", "green")
    else:
        print_in_color(f"{file_name}.jpg has been downloaded already.\n", "green")

def video_downloader(shortcode, videos_path):
    """ Downloads videos """
    videos = requests.get(f"https://www.instagram.com/p/{shortcode}/?__a=1")
    video_url = videos.json()['graphql']['shortcode_media']['video_url']
    file_name = videos.json()['graphql']['shortcode_media']['taken_at_timestamp']
    download_path = f"{videos_path}\\{file_name}.mp4"
    if not os.path.exists(download_path):
        print_in_color(f"Downloading {file_name}.mp4...........", "yellow")
        urllib.request.urlretrieve(video_url, download_path)
        print_in_color(f"{file_name}.mp4 downloaded.\n", "green")
    else:
        print_in_color(f"{file_name}.mp4 has been downloaded already.\n", "green")

def sidecar_downloader(shortcode, images_path, videos_path):
    """ Downloads images and videos from posts containing more than one pictures or videos """
    r = requests.get(f"https://www.instagram.com/p/{shortcode}/?__a=1")
    num = 1
    for edge in r.json()['graphql']['shortcode_media']['edge_sidecar_to_children']['edges']:
        is_video = edge['node']['is_video']
        if not is_video:
            display_url = edge['node']['display_url']
            file_name = r.json()['graphql']['shortcode_media']['taken_at_timestamp']
            download_path = f"{images_path}\\{file_name}_{num}.jpg"
            if not os.path.exists(download_path):
                print_in_color(f"Downloading {file_name}_{num}.jpg...........", "yellow")
                urllib.request.urlretrieve(display_url, download_path)
                print_in_color(f"{file_name}_{num}.jpg downloaded.\n", "green")
            else:
                print_in_color(f"{file_name}_{num}.jpg has been downloaded already.\n", "green")
        else:
            video_url = edge['node']['video_url']
            file_name = r.json()['graphql']['shortcode_media']['taken_at_timestamp']
            download_path = f"{videos_path}\\{file_name}_{num}.mp4"
            if not os.path.exists(download_path):
                print_in_color(f"Downloading {file_name}_{num}.mp4...........", "yellow")
                urllib.request.urlretrieve(video_url, download_path)
                print_in_color(f"{file_name}_{num}.mp4 downloaded.\n", "green")
            else:
                print_in_color(f"{file_name}_{num}.mp4 has been downloaded already.\n", "green")
        num += 1

def main(account_json_info, path):
    """ Runs methods that download photos/videos from the user """
    init()
    r = requests.get(account_json_info)
    user_id = r.json()['graphql']['user']['id']
    end_cursor = ''
    next_page = True
    images_path = f"{path}\\Images"
    videos_path = f"{path}\\Videos"
    if not os.path.exists(path):
        os.makedirs(path)
        if not os.path.exists(images_path):
            os.makedirs(images_path)
        if not os.path.exists(videos_path):
            os.makedirs(videos_path)
        print_in_color("User Folder Created!\n", "magenta")
    else:
        print_in_color("User Folder Has Been Created Already!\n", "magenta")

    while next_page:
        r = requests.get('https://www.instagram.com/graphql/query/',
                params={
                    'query_id': '17880160963012870',
                    'id': user_id,
                    'first': 12,
                    'after': end_cursor
                }
            )
        graphql = r.json()['data']
        for edge in graphql['user']['edge_owner_to_timeline_media']['edges']:
            __typename = edge['node']['__typename']
            if __typename == 'GraphImage':
                image_downloader(edge, images_path)
            elif __typename == 'GraphVideo':
                shortcode = edge['node']['shortcode']
                video_downloader(shortcode, videos_path)
            elif __typename == 'GraphSidecar':
                shortcode = edge['node']['shortcode']
                sidecar_downloader(shortcode, images_path, videos_path)

        end_cursor = graphql['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        next_page = graphql['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']
        time.sleep(10)
    deinit()

if __name__ == '__main__':
    print('\n\n')
    init(autoreset=True)
    print_in_color('Instagram Media Downloader'.center(os.get_terminal_size().columns, '-'), "cyan")
    deinit()

    PARSER = argparse.ArgumentParser(description='Download Instagram Images and Videos from a User\'s Profile Page')
    PARSER.add_argument('-u', '--user', dest='username', required=True, help='Username on Instagram')
    PARSER.add_argument('-p', '--path', dest='path', required=True, help='Root path where downloaded Instagram Media is saved')
    ARGS = PARSER.parse_args()

    #Insert username into link
    ACCOUNT_JSON_INFO = f"https://www.instagram.com/{ARGS.username}/?__a=1"
    ARGS.path += f"\\{ARGS.username}"
    main(ACCOUNT_JSON_INFO, ARGS.path)




-----------------------------------------------Instagram Media Downloader-----------------------------------------------


usage: ipykernel_launcher.py [-h] -u USERNAME -p PATH
ipykernel_launcher.py: error: the following arguments are required: -u/--user, -p/--path


SystemExit: 2

C:\Users\yaraz\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
import instloader


ModuleNotFoundError: No module named 'instloader'